In [2]:
import base64
import boto3
import io
import json
from PIL import Image
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorchModel


from config import IMAGE_TEST

In [55]:
role = get_execution_role()

pytorch_model = PyTorchModel(model_data='s3://weights-captcha/model.tar.gz', role=role,
                             entry_point='inference.py', framework_version="1.7.1",
                             py_version="py3",
                             )

predictor = pytorch_model.deploy(instance_type='ml.t3.medium', initial_instance_count=1)
endpoint = predictor.endpoint
endpoint

-------!

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'pytorch-inference-2021-10-15-03-29-08-633'

In [60]:
image = Image.open(IMAGE_TEST).convert("RGB")
#bytesarray = bytes(image.tobytes())
#image_data = Image.open(io.BytesIO(image.tobytes()))
#image.show()

buffered = io.BytesIO()
image.save(buffered, format="JPEG")
img_str = base64.b64encode(buffered.getvalue())

payload = {"image": img_str.decode()}

payload = json.dumps(payload)

runtime = boto3.client('sagemaker-runtime')

response = runtime.invoke_endpoint(
    EndpointName=endpoint,
    ContentType='application/json',
    Body=payload)

result = json.loads(response['Body'].read().decode()) 


In [61]:
print(result)

[[21.167724609375, -4.827481746673584, -3.0279018878936768, -4.160379886627197, -5.018439769744873, -1.498039960861206, -4.156457901000977, -4.682018280029297, -2.261157751083374, -3.038466215133667, -3.278585910797119, -8.060171127319336, -0.9796529412269592, -2.676953077316284, -5.099091053009033, -3.564992666244507, -2.206869125366211, -4.332683563232422, -4.04838228225708, -2.747683525085449, -3.823640823364258, -4.809587001800537, -1.8433430194854736, -2.6118063926696777, -4.019082069396973, -4.21744441986084, -3.994037389755249, -2.758504867553711, -3.3628532886505127, -1.86456298828125, -2.692321538925171, -3.0455448627471924, -3.3005998134613037, -3.282867670059204, -2.0690176486968994, -2.7778306007385254, -3.6720173358917236, -2.216061592102051]]


### Eliminar Endpoint

In [1]:
predictor.delete_endpoint()

NameError: name 'predictor' is not defined